# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,296.06,87,100,7.91,PN,1699936302
1,1,masterton,-40.9597,175.6575,290.86,63,70,5.91,NZ,1699936302
2,2,cloverdale,38.8055,-123.0172,286.89,84,52,0.45,US,1699936303
3,3,port mathurin,-19.6833,63.4167,298.69,77,1,3.46,MU,1699936303
4,4,stanley,54.8680,-1.6985,281.92,92,20,3.09,GB,1699936304


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
# Configure the map plot
map_plot_2 = city_data_df.hvplot.points("Lng",
                                        "Lat",
                                        geo =True,
                                        tiles = "EsriImagery",
                                        frame_width = 800,
                                        frame_height = 600,
                                        size = "Humidity",
                                        scale = 0.5,                                      
                                       
                                       )

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
#Define the criteria (e.e, max temprature, humidity, cloudiness, wind speed)
#filter the DataFrame based on the criteria
city_data_df = city_data_df[
    (city_data_df["Max Temp"] <= 300) & (city_data_df["Cloudiness"] == 40) & (city_data_df["Wind Speed"]<= 19)]

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,hawaiian paradise park,19.5933,-154.9731,295.88,94,40,2.06,US,1699936204
9,9,taiohae,19.5933,-154.9731,295.88,94,40,2.06,US,1699936204
35,35,longyearbyen,78.2186,15.6401,262.06,73,40,2.57,SJ,1699936317
46,46,avarua,-21.2078,-159.7750,296.18,64,40,3.09,CK,1699936120
83,83,geraldton,-28.7667,114.6000,299.38,61,40,5.14,AU,1699936270
129,129,haiku-pauwela,20.9219,-156.3051,298.17,68,40,6.69,US,1699936125
196,196,hilo,19.7297,-155.0900,296.00,92,40,2.06,US,1699936301
234,234,ivolginsk,51.7492,107.2837,268.96,68,40,1.00,RU,1699936391
335,335,nishinoomote,30.7333,131.0000,289.46,63,40,4.12,JP,1699936436
432,432,souillac,-20.5167,57.5167,298.36,78,40,3.60,MU,1699936477


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df =city_data_df[["City","Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,hawaiian paradise park,US,19.5933,-154.9731,94,
9,taiohae,US,19.5933,-154.9731,94,
35,longyearbyen,SJ,78.2186,15.6401,73,
46,avarua,CK,-21.2078,-159.7750,64,
83,geraldton,AU,-28.7667,114.6000,61,
129,haiku-pauwela,US,20.9219,-156.3051,68,
196,hilo,US,19.7297,-155.0900,92,
234,ivolginsk,RU,51.7492,107.2837,68,
335,nishinoomote,JP,30.7333,131.0000,63,
432,souillac,MU,-20.5167,57.5167,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000 # Set to 10,000 meters
 
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row ["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params = params)
    
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")


# Display sample data
hotel_df

Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
taiohae - nearest hotel: No hotel found
longyearbyen - nearest hotel: The Vault
avarua - nearest hotel: Paradise Inn
geraldton - nearest hotel: Ocean Centre Hotel
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
hilo - nearest hotel: Dolphin Bay Hotel
ivolginsk - nearest hotel: No hotel found
nishinoomote - nearest hotel: ゼウスハウス
souillac - nearest hotel: Hotel SkylineInn
happy valley-goose bay - nearest hotel: Hotel North Two
macau - nearest hotel: 富華粵海酒店 Fu Hua Guang Dong Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
8,hawaiian paradise park,US,19.5933,-154.9731,94,No hotel found
9,taiohae,US,19.5933,-154.9731,94,No hotel found
35,longyearbyen,SJ,78.2186,15.6401,73,The Vault
46,avarua,CK,-21.2078,-159.7750,64,Paradise Inn
83,geraldton,AU,-28.7667,114.6000,61,Ocean Centre Hotel
129,haiku-pauwela,US,20.9219,-156.3051,68,Inn At Mama's Fish House
196,hilo,US,19.7297,-155.0900,92,Dolphin Bay Hotel
234,ivolginsk,RU,51.7492,107.2837,68,No hotel found
335,nishinoomote,JP,30.7333,131.0000,63,ゼウスハウス
432,souillac,MU,-20.5167,57.5167,78,Hotel SkylineInn


In [7]:
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,hawaiian paradise park,US,19.5933,-154.9731,94,No hotel found
9,taiohae,US,19.5933,-154.9731,94,No hotel found
35,longyearbyen,SJ,78.2186,15.6401,73,The Vault
46,avarua,CK,-21.2078,-159.7750,64,Paradise Inn
83,geraldton,AU,-28.7667,114.6000,61,Ocean Centre Hotel
129,haiku-pauwela,US,20.9219,-156.3051,68,Inn At Mama's Fish House
196,hilo,US,19.7297,-155.0900,92,Dolphin Bay Hotel
234,ivolginsk,RU,51.7492,107.2837,68,No hotel found
335,nishinoomote,JP,30.7333,131.0000,63,ゼウスハウス
432,souillac,MU,-20.5167,57.5167,78,Hotel SkylineInn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
# Configure the map
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols=['Hotel Name', 'Country']  # Add additional hover columns
)

# Display the map plot
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)